In [95]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [103]:
#upload raw data
final_demo=pd.read_csv('../Project_2/data/raw/df_final_demo.txt')
final_experiment_clients=pd.read_csv('../Project_2/data/raw/df_final_experiment_clients.txt')
final_web_data_pt1=pd.read_csv('../Project_2/data/raw/df_final_web_data_pt_1.txt')
final_web_data_pt2=pd.read_csv('../Project_2/data/raw/df_final_web_data_pt_2.txt')

### Context & Data insides
You are a newly employed data analyst in the Customer Experience (CX) team at Vanguard, the US-based investment management company. You’ve been thrown straight into the deep end with your first task. Before your arrival, the team launched an exciting digital experiment, and now, they’re eagerly waiting to uncover the results and need your help!

# Dataset

1 - Client Profiles (df_final_demo):
    - client_id -> unique number. (int)
    - clnt_tenure_yr -> Represents how many years the client has been with Vanguard. (int)
    - clnt_tenure_mnth -> Represents how many mounths the client has been with Vanguard. (int)
    - clnt_age -> Indicate the age of the client. (float)
    - gendr -> Specifies the client's gender. (int)
    - num_accts -> Denotes the number of accounts the client holds particular client. (int)
    - Bal -> Gives the total balance spread across all accounts for a particular client. (float)
    - call_6_mnth -> Records the number of times the client reached out a call in the past six months. (int)
    - logons_6_mnth -> Reflects the frequency with which the client logged onto Vanguard's platform. (int)

2 -  Digital Footprints (df_final_web_data) - a detailed trace of client interactions online:
    - client_id -> unique number. (int)
    - visitor_id -> A unique ID for each client-device combination. (str)
    - visit_id -> A unique ID for each web visit/session. (str)
    - process_step -> Marks each step in the digital process. (str)

3- Experiment Roster(df_final_experiment_clients) - A list revealing which clients were part of the grand experiment:
    - client_id -> unique number. (int)
    - Variation -> if it was part of control group or test group. (str)
        control group: Clients interacted with Vanguard's traditional online process.
        Test group: Clents experienced the new, spruced-up digital interface.

    Hipothesis:
    Vanguard believed that a more intuitive and modern User Interface (UI), coupled with timely in-context prompts (cues, messages, hints, or instructions provided to users directly within the context of their current task or action), could make the online process smoother for clients.

    Question:
    Would these changes encourage more clients to complete the process?

## Cleaning

In [63]:
# Client Profiles
final_demo.dropna(thresh=7,inplace=True)
# Digital Footprints
final_web_data_final= pd.concat([final_web_data_pt1,final_web_data_pt2])
final_web_data_final.drop_duplicates(inplace=True)
# Experiment Roster
final_experiment_clients.dropna(inplace=True) 

In [105]:
final_demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


In [104]:
final_demo[final_demo['clnt_age']<=final_demo['clnt_tenure_yr']] # I will say to drop si pertenencen o no al experimentos

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
108,7449147,27.0,330.0,26.5,F,2.0,215247.66,3.0,6.0
846,4267612,28.0,345.0,28.0,F,2.0,32358.76,3.0,6.0
970,4955883,29.0,349.0,26.0,F,2.0,37122.69,2.0,5.0
1138,1330615,23.0,285.0,18.5,M,2.0,35553.33,3.0,6.0
1194,4286237,21.0,257.0,17.5,F,2.0,32577.78,5.0,8.0
...,...,...,...,...,...,...,...,...,...
70420,6911362,30.0,361.0,23.0,F,2.0,51142.87,3.0,3.0
70437,699213,23.0,286.0,17.0,M,2.0,29734.42,6.0,6.0
70439,6672629,24.0,299.0,23.0,U,4.0,91746.29,4.0,4.0
70479,9696505,20.0,250.0,18.5,F,2.0,15010.15,2.0,2.0


In [106]:
final_demo['gendr'].value_counts() # replace X by U

gendr
U    24122
M    23724
F    22746
X        3
Name: count, dtype: int64

In [107]:
final_demo['num_accts'].value_counts() # does it make sense to have 8, 7 6 accounts ? --> check outlier probably

num_accts
2.0    55497
3.0    12529
4.0     2241
5.0      284
6.0       33
7.0        8
1.0        2
8.0        1
Name: count, dtype: int64

In [108]:
final_demo['logons_6_mnth'].value_counts() # why not 0.

logons_6_mnth
9.0    12350
3.0    10243
4.0    10220
7.0     9893
5.0     9254
6.0     7878
8.0     4810
2.0     3009
1.0     2938
Name: count, dtype: int64

In [101]:
print(f"average {final_demo['bal'].mean().__round__(2)},median {final_demo['bal'].median()}, mode   {final_demo['bal'].mode()[0]}")

average 147445.24,median 63332.9, mode   31188.62


In [102]:
# 2 - Measures of Dispersion:
print(f"var   {final_demo['bal'].var()}")
print(f"std   {final_demo['bal'].std()}")
print(f"range   {(final_demo['bal'].max()-final_demo['bal'].min())}")
print(f"q25   {final_demo['bal'].quantile([0.25, 0.5, 0.75,.85])}")

var   90907500113.93842
std   301508.70653090335
range   16306250.73
q25   0.25     37346.835
0.50     63332.900
0.75    137544.905
0.85    224843.889
Name: bal, dtype: float64


In [78]:
final_demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


In [30]:
final_demo['clnt_age'].unique()

array([60.5, 58. , 32. , 49. , 33. , 30.5, 58.5, 57.5, 67.5, 54.5, 34.5,
       51.5, 62.5, 36. , 74. , 32.5, 31. , 25.5, 42.5, 42. , 30. , 55. ,
       38. , 68.5, 52.5, 54. , 63. , 48. , 66.5, 68. , 65. , 51. , 46. ,
       63.5, 23. , 26.5, 56.5, 76.5, 27.5, 39.5, 25. , 27. , 60. , 66. ,
       39. , 56. , 40. , 52. , 31.5, 43.5, 35. , 19. , 64.5, 64. , 22.5,
       33.5, 36.5, 61. , 40.5, 72. , 44.5, 28.5, 59.5, 45. , 18. , 78. ,
       50.5, 29. , 37.5, 57. , 70. , 74.5, 22. , 50. , 53. , 62. , 65.5,
       29.5, 28. , 69.5, 35.5, 71.5, 34. , 38.5, 69. , 43. , 83.5, 21.5,
       47. , 61.5, 73.5, 71. , 59. , 21. , 44. , 83. , 47.5, 72.5, 76. ,
       78.5, 55.5, 67. , 41.5, 20.5, 75. , 24. , 79. , 26. , 41. , 23.5,
       49.5, 46.5, 45.5, 48.5, 24.5, 75.5, 80. , 73. , 87. , 19.5, 53.5,
       18.5, 70.5, 80.5, 86. , 37. , 85. , 17.5, 82. , 20. , 77.5, 77. ,
       79.5, 88.5, 96. , 81. , 81.5, 88. , 84. ,  nan, 94.5, 89. , 92. ,
       82.5, 91.5, 85.5, 95.5, 84.5, 90. , 89.5, 93

In [64]:
final_demo[final_demo['clnt_age'].isna()]

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
9583,4666211,8.0,106.0,NaN,F,2.0,42550.55,4.0,7.0
